In [25]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [26]:
url_inicial = 'https://historicas.unam.mx/publicaciones/revistas/moderna/ehmcnums.html'
url_root = 'https://historicas.unam.mx/publicaciones/revistas/moderna/ehmcnums.html'
r=requests.get(url_inicial)

In [27]:
soup = BeautifulSoup(r.text, 'html.parser')

In [28]:
box = soup.find('div', class_='table-responsive')
volumen=box.findAll('td', width='50%')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['http://dx.doi.org/10.22201/iih.24485004e.2020.59',
 'http://dx.doi.org/10.22201/iih.24485004e.2019.58',
 'https://historicas.unam.mx/publicaciones/revistas/moderna/vols/ehmc57/M57.html']

In [29]:
def get_url_items(sopa, url): 
    box = soup.find('div', class_='table-responsive')
    volumen=box.findAll('td', width='50%')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [30]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links)

Estoy en la pagina https://historicas.unam.mx/publicaciones/revistas/moderna/ehmcnums.html


In [31]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

3

In [32]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [42]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Estudios de Historia Moderna y Contemporánea de México'
    a='IV. Humanidades y Ciencias de la Conducta'
    tem='Historia de México'
    url='https://historicas.unam.mx/publicaciones/revistas/moderna/moderna.html'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('h2', class_='title').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('a', class_='cover').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('section', class_='container issue__issue-data').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [43]:
list_scraper=list_scraper[0:3]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2


In [44]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Link inicial,Link articulo
0,Estudios de Historia Moderna y Contemporánea d...,IV. Humanidades y Ciencias de la Conducta,Historia de México,None,None,https://historicas.unam.mx/publicaciones/revis...,None
1,Estudios de Historia Moderna y Contemporánea d...,IV. Humanidades y Ciencias de la Conducta,Historia de México,None,None,https://historicas.unam.mx/publicaciones/revis...,None
2,Estudios de Historia Moderna y Contemporánea d...,IV. Humanidades y Ciencias de la Conducta,Historia de México,None,None,https://historicas.unam.mx/publicaciones/revis...,None


In [ ]:
df_catalogo.to_csv('Revista81.csv', index=False)